AutoCop, prueba de concepto del Observatorio de Contenidos Audiovisuales (OCA), financiado por la Fundación General de la Universidad de Salamanca [Plan TCUE 2015-2017 Fase 2]. 
Investigador principal:
Carlos Arcila Calderón
Investigadores:
Félix Ortega, Javier Amores, Sofía Trullenque, Miguel Vicente, Mateo Álvarez, Javier Ramírez

# Twitter streaming sentiment analyzer with Spark

## 1. Description of the tool

The development is a multipart tool to make sentiment analysis over the twitter stream with Spark Streaming API, written in python and with integration with kafka for the streaming and mongoDB for storage.
This development is implemented in Jupyter notebooks as well as in python scripts that use Pyspark

The tool is divided in five parts: the data gathering and filtering, with kafka, the data processing, with Spark, the data storage, in mongoDB, the data visualization, with Jupyter Notebooks script and Bokeh library, and the model training scripts, in jupyter notebooks.

## 1.1 Components

This tool is comprised by the following parts:
 - Kafka producer with the tweepy library that receives data from Twitter API
 - Spark Streaming app to classify the tweets and store them into a mongoDB database
 - Visualization script for the processed data that reads from the database
 - Jupyter notebook file to train the models
 
The repository contains Jupyter notebooks to execute the code in local and also scripts to execute the analysis remotely in a Cloud environment

### Standalone version
This application can be run either in standalone mode, in a single node or in a distributed mode (AWS deployment instructions included). The standalone version will run through the Notebooks via Jupyter or with the scripts provided.

## Prerequisites
For the standalone, the following prerequisites are required:
 - Kafka server (0.8)
 - Zookeeper server
 - Spark 2.1 & SPARK_HOME, spark libraries: spark-streaming-kafka-0-8-assembly_2.11:2.1.0, mongo-spark-connector_2.10:2.0.0
 - Python 3.5 and libraries: tweepy, kafka, pyspark, findspark, bokeh, pymongo

## Setup kafka server
### Download kafka

The first step is to download Apache Kafka, the version this script uses is Scala 2.11  - kafka_2.11-0.8.2.2.tgz (asc, md5), the download link is the following: http://mirrors.gigenet.com/apache/kafka/0.8.2.2/kafka_2.11-0.8.2.2.tgz

Once downloaded and unzipped the folder to a desired location, setup configuration for zookeeper and kafka services

### Zookeeper and Kafka configuration

The configuration files are stored in kafka_folder/config/
    Zookeeper: zookeeper.properties
    Kafka: server.properties
    
Here are some example files:

zookeeper.properties
server.properties

### Start Zookeeper service

To run the kafka server, a zookeeper instance is necessary, you can use one of yours or the one included in the kafka folder. To start the zookeeper: **kafka_folder/bin/zookeeper-server-start.sh config/zookeeper.properties**

### Start Kafka Server

Once Zookeeper is running, we can start kafka server with this command: **kafka_folder/bin/kafka-server-start.sh config/server.properties**

For now we have Zookeeper and a Kafka server running, to start recieving and processing twitter streaming, we have to start a kafka producer from the twitter stream and then read the stream from the broker through spark streaming using the Kafka Stream Class.

### Start Kafka producer

To start the Kafka producer, just execute the following cells.
These cells use the tweepy library to connect to the Twitter API, to do so, credentials for the API must be provided.

### Parameters

Some parameters have to be configured.

First of all the Twitter API parameters, which require auth params as well as a hashtag, then we have to configure kafka parameters.
Kafka will require the location of the kafka server, the upodate frequency of the producer (frequency to gather tweets from the API), and the topic to write on, this parameter is taken from the hashtag, and the kafka topic created for each hashtag will have the name of each one.

## Follow the logs
After launching the kafka application, the tweets can be seen introducing this command in the terminal, in the kafka folder: **bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic BigData --from-beginning**
where BigData is the topic to be read
## Kafka consumer
Once the previous infrastructure is running, we have a kafka producer connected to the twitter API. To analyze the tweets with Spark, just launch the twitter-kafka-consumer-spark-streaming.ipynb notebook (the models and the words have to have been generated before with the twitter-spark-model-training.ipynb notebook)

To execute the notebooks, start the notebook server and run the cells of every notebook with the desired configuration.

# Deploy instructions for AWS

## Prerequisites for the deployment in AWS environment

To deploy the tool in a remote server, some steps have to be considered:


### Provisioning machines

First of all an EC2 machine has to be provisioned, in order to run all the non-spark services: kafka, zookeeper and mongodb. The requirements of this machine will vary from each application, but in general, a t2.medium or t2.large should be enough. This instance shall have a S3 volume attached in order to store the data from the database. It is also necessary to configure the ports for the services, via security group. Kafka uses 9092, zookeeper uses 2181, and mongodb uses 32768.

An EMR cluster must be deployed to execute the spark jobs in a distributed environment. Trials with the smallest cluster have been made, as the application does not require large ammount of processing capacity.

### Configuring machines

Some scripts have been generated in order to preconfigure the machines:
 - **configure_instance.sh** contains commands to be executed in the EC2 instance, and basically downloads kafka, and installs python 3.5 and docker, after all it starts a mongodb instance with a docker image. 
 - It is also necessary to modify the server.properties and producer.properties files in the kafka conf folder, in order to use your machine's parameters
 - **configure_workers.sh** is a bash script that configure the spark driver and workers and have to be executed in all machines of the spark cluster in order to have all de dependencies installed in the machines.
 - Appart from the machine's configuration, some files have to be uploaded to the HDFS node running on the spark master, these files are the trained models and the word dictionary that the algorithms will use to convert text into instances

## Run the kafka consumer

Once the EC2 machine is configured, start the zookeeper server with the command showed in the standalone version, and also the kafka server. The logs can also be seen with the command shown on the standalone version.

With these services running, modify the **kafka-producer.py** file in order to set your twitter app credentials, the hashtag and the kafka address (connection string), and execute it with python 3.5

## Run the Spark job

Once the kafka producer is running and the spark environment is set, modify the file **kafka-consumer.py** in order to set your configuration and execute it with a spark-submit command, including the necessary packages, via "--packages" or "--jars" attributes.